<a href="https://colab.research.google.com/github/hy05sprsea/KIKAGAKU-Tutorial/blob/master/%E3%83%95%E3%82%A1%E3%82%A4%E3%83%B3%E3%83%81%E3%83%A5%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf

In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8329648957079839188
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7306918986385454606
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 6385008339127079427
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14648777152
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10100993677579190258
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


In [6]:
train, test = tf.keras.datasets.cifar10.load_data()

In [7]:
# 画像の情報
train[0].shape, test[0].shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [8]:
# ラベルの情報
train[1].shape, test[1].shape

((50000, 1), (10000, 1))

In [9]:
# 学習用データセットとテスト用データセットに対して正規化
x_train = train[0] / 255
x_test = test[0] / 255

In [10]:
t_train = train[1]
t_test = test[1]

In [11]:
t_train.dtype

dtype('uint8')

In [12]:
# 32bit にキャスト
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
t_train, t_test = t_train.astype('int32'), t_test.astype('int32')

In [13]:
import os, random

def reset_seed(seed=0):
    os.environ['PYTHONHASHSEED'] = '0'
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [14]:
from tensorflow.keras import models, layers

reset_seed(0)

# モデルの構築
model = models.Sequential([
    # 特徴量抽出
    layers.Conv2D(filters=3, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPool2D(pool_size=(2, 2)),
    # ベクトル化
    layers.Flatten(),
    # 識別
    layers.Dense(100, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 3)         84        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 3)         0         
_________________________________________________________________
flatten (Flatten)            (None, 675)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               67600     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 68,694
Trainable params: 68,694
Non-trainable params: 0
_________________________________________________________________


In [16]:
# optimizerの設定
optimizer = tf.keras.optimizers.Adam(lr=0.01)

# モデルのコンパイル
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
# モデルの学習
batch_size = 4096
epochs = 30

# 学習の実行
history = model.fit(x_train, t_train,
                batch_size=batch_size,
                epochs=epochs, 
                validation_data=(x_test, t_test))


Epoch 1/30
13/13 [==============================] - 1s 65ms/step - loss: 2.1485 - accuracy: 0.2161 - val_loss: 1.9056 - val_accuracy: 0.3299
Epoch 2/30
13/13 [==============================] - 1s 43ms/step - loss: 1.8599 - accuracy: 0.3522 - val_loss: 1.7672 - val_accuracy: 0.3799
Epoch 3/30
13/13 [==============================] - 1s 45ms/step - loss: 1.6805 - accuracy: 0.4035 - val_loss: 1.6401 - val_accuracy: 0.4145
Epoch 4/30
13/13 [==============================] - 1s 42ms/step - loss: 1.5715 - accuracy: 0.4382 - val_loss: 1.5540 - val_accuracy: 0.4433
Epoch 5/30
13/13 [==============================] - 1s 44ms/step - loss: 1.4921 - accuracy: 0.4687 - val_loss: 1.5071 - val_accuracy: 0.4581
Epoch 6/30
13/13 [==============================] - 1s 44ms/step - loss: 1.4463 - accuracy: 0.4848 - val_loss: 1.4698 - val_accuracy: 0.4753
Epoch 7/30
13/13 [==============================] - 1s 45ms/step - loss: 1.4003 - accuracy: 0.5018 - val_loss: 1.4739 - val_accuracy: 0.4743
Epoch 8/30
13

# **ファインチューニング**

In [18]:
from tensorflow.keras.applications import resnet, VGG16

# 学習済みモデルをインスタンス化
base_model = VGG16(input_shape=(244, 244, 3),
                   include_top=False, weights='imagenet')

In [19]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 244, 244, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 244, 244, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 244, 244, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 122, 122, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 122, 122, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 122, 122, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 61, 61, 128)       0     

In [20]:
# ランダムにデータを取得する
train_choice = np.random.randint(low=0, high=50000, size=10000)
test_choice = np.random.randint(low=0, high=10000, size=5000)

In [21]:
# データの準備
x_train = train[0][train_choice]
x_test = test[0][test_choice]
t_train = train[1][train_choice].astype('int32') 
t_test = test[1][test_choice].astype('int32') 

In [24]:
_train, _test = [], []

for img in x_train:
  _train.append(cv2.resize(src=img, dsize=(244, 244)))
for img in x_test:
  _test.append(cv2.resize(src=img, dsize=(244, 244)))

**セッションがクラッシュしました**